In [1]:
import numpy as np 
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import string
import re


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
summarize = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
class chatbot:
    def __init__(self):
        self.doc: pd.DataFrame = pd.read_excel("C:/Users/Yash/Desktop/NLP project/Data.xlsx")
        temp = list(self.doc["text"])
        
        #lower case conversion
        for i in range(len(temp)):
            temp[i] = temp[i].lower()
        
        #stopword removal
        stop_words = stopwords.words('english')
        for i in range(len(temp)):
            empty = "".join([word+" " for word in temp[i].split() if word not in stop_words])
            temp[i] = empty
            
        
        #lemmatazing
        porter = PorterStemmer()
        for i in range(len(temp)):
            empty = ""
            for j in temp[i].split():
                empty += porter.stem(j)+" "
            temp[i] = empty
        
        #sentence tokenizor
        for i in range(len(temp)):
            temp[i] = sent_tokenize(temp[i])
            
        #removing punctions
        for i in range(len(temp)):
            t = []
            for j in range(len(temp[i])):
                text_p = "".join([char for char in temp[i][j] if char not in string.punctuation])
                t.append(text_p)
            temp[i] = t
        
        #storing pre-processed data
        self.doc["pre_processed"] = temp
        
        #Converting to embedding
        self.doc["embedding"] = self.doc["pre_processed"].apply(lambda x: model.encode(x, convert_to_tensor=True))
        self.temp: pd.DataFrame;
        
    def QA(self, flag):
        print("\n\n")
        Q = str(input("Enter your Question >>> "))
        
        Q = Q.lower()
        end = ['goodbye', 'bye', 'see you later', 'talk to you soon', 'take care']
        
        if Q in end:
            return True
        
        stop_words = stopwords.words("english")
        porter = PorterStemmer()
        empty = ""
        for word in Q.split():
            if word not in stop_words:
                empty += porter.stem(word) + " "
                
        Q_embedding = model.encode(Q, convert_to_tensor=True)
#         print(Q_embedding)

        sim = []
        for i in self.doc["embedding"]:
            maxx = 10e-9
            for j in i:
                comm = util.cos_sim(Q_embedding, j)
                if(comm>maxx):
                    maxx = comm
            sim.append(maxx)
            
        self.temp: pd.DataFrame = pd.DataFrame(self.doc)
        self.temp["similarity"] = sim
        self.temp = self.temp.sort_values("similarity", ascending=False) 
#         self.temp = self.temp.reset_index()

        
        context = self.temp.loc[0, "text"]
        summary = summarize(context, max_length=200, min_length=100, do_sample=False)[0]['summary_text']
        print("bot >> according to best match: \n", summary)
            
            
        
        return False 

bot = chatbot()

In [8]:
flag = False
while not flag:
    flag = bot.QA(flag)




Enter your Question >>> what is machine learning
bot >> according to best match: 
  Machine Learning tutorial covers basic and advanced concepts, specially designed to cater to both students and experienced working professionals . This is the field of study that gives computers the capability to learn without being explicitly programmed. It is one of the most exciting technologies that one would have ever come across. It gives the computer that makes it more similar to humans: The ability to learn. The tutorial helps you gain a solid introduction to the fundamentals of machine learning and explore a wide range of techniques, including supervised, unsupervised, and reinforcement learning.



Enter your Question >>> AI
bot >> according to best match: 
  Machine Learning tutorial covers basic and advanced concepts, specially designed to cater to both students and experienced working professionals . This is the field of study that gives computers the capability to learn without being ex